# 📝 ScratchpadTools - Agent Memory System

```mermaid
%%{init: {'theme':'base', 'themeVariables': { 'primaryColor':'#F1C40F', 'primaryTextColor':'#000', 'primaryBorderColor':'#F39C12', 'lineColor':'#E67E22', 'secondaryColor':'#3498DB', 'tertiaryColor':'#27AE60', 'fontSize':'16px'}}}%%
graph TB
    A[👤 User: Remember this] --> B[🤖 Agent]
    B --> C{✍️ WriteToScratchPad}
    C --> D[💾 Memory Storage]
    
    E[👤 User: What did I say?] --> F[🤖 Agent]
    F --> G{📖 ReadFromScratchPad}
    G --> D
    D --> H[📤 Retrieved Memory]
    
    style A fill:#3498DB,stroke:#2980B9,color:#fff
    style C fill:#F1C40F,stroke:#F39C12,color:#000
    style D fill:#E67E22,stroke:#D35400,color:#fff
    style G fill:#F39C12,stroke:#E67E22,color:#fff
    style H fill:#27AE60,stroke:#229954,color:#fff
```

## 📚 Learning Objectives

1. ✅ **WriteToScratchPadTool**: Store information in agent memory
2. ✅ **ReadFromScratchPadTool**: Retrieve stored information
3. ✅ Building **stateful conversational agents**
4. ✅ Managing **conversation context and history**
5. ✅ **Persistent notes** across agent executions

---

## 🎯 What are ScratchpadTools?

**ScratchpadTools** provide memory capabilities for agents:
- 💾 **WriteToScratchPadTool**: Save information for later
- 📖 **ReadFromScratchPadTool**: Retrieve saved information
- 🔄 **Persistent Memory**: Information survives across agent calls
- 📝 **Context Tracking**: Maintain conversation state

**Use Cases**:
- 💬 Multi-turn conversations
- 📊 Progressive data collection
- 🎯 Personalized interactions
- 🔄 Workflow state management

---

## Step 1: Import Libraries

In [1]:
import sys
import json

sys.path.append('..')
from agent_helpers import (
    get_os_client,
    create_flow_agent,
    execute_agent,
    cleanup_resources
)

print("✅ Libraries imported!")

✅ Libraries imported!


## Step 2: Initialize Client

In [2]:
client = get_os_client()
print(f"✅ Connected to: {client.info()['cluster_name']}")

✅ Connected to: docker-cluster


## Step 3: Create Agent with Write Capability

In [3]:
# Agent with WriteToScratchPadTool
write_tools = [{
    "type": "WriteToScratchPadTool",
    "parameters": {
        "return_history": True
    }
}]

write_agent_id = create_flow_agent(
    client, "Memory_Writer_Agent",
    "Agent that writes information to scratchpad memory",
    write_tools
)
print(f"✅ Write agent created: {write_agent_id}")

   Registering flow agent: Memory_Writer_Agent...
   ✓ Agent registered: kVuSiZsBLQ1mV2UN-ymm
✅ Write agent created: kVuSiZsBLQ1mV2UN-ymm


## Step 4: Test Writing to Memory

In [4]:
# Write user preferences
parameters = {
    "notes": "User prefers: dashboard views, daily email reports, data in PST timezone"
}

print("✍️ Writing to scratchpad...")
response = execute_agent(client, write_agent_id, parameters)
print(json.dumps(response, indent=2))

✍️ Writing to scratchpad...
{
  "inference_results": [
    {
      "output": [
        {
          "name": "response",
          "result": "Scratchpad updated. Full content:\n- User prefers: dashboard views, daily email reports, data in PST timezone"
        }
      ]
    }
  ]
}


## Step 5: Write More Information

## ⚠️ Important: Understanding Scratchpad Memory Scope

**Key Limitation**: The scratchpad memory is **session-based** and **agent-isolated**:
- ✅ Memory persists **within a single agent execution**
- ❌ Memory does **NOT persist** across different `execute_agent()` calls
- ❌ Different agents have **separate** scratchpad memories

This means:
- When you call `execute_agent()` multiple times, each call starts with a fresh scratchpad
- The scratchpad is designed for **intra-workflow** memory, not **inter-execution** persistence

For **true persistent memory**, you would need:
- External database storage
- Custom memory management system  
- State management in your application layer

In [12]:
# Add conversation context - NOTE: return_history shows accumulated notes within THIS agent's memory
parameters = {
    "notes": "Previous query: Sales analysis for Q3 2025. User interested in regional breakdown.",
    "return_history": True  # Returns all notes stored in THIS write agent's memory
}

print("✍️ Adding more context to write agent's memory...")
response = execute_agent(client, write_agent_id, parameters)
print(json.dumps(response, indent=2))

# ⚠️ Important: Each agent has its own isolated scratchpad memory!
# The write_agent_id and read_agent_id have SEPARATE memories
print("\n⚠️ Note: Write and Read agents have separate memory spaces!")

✍️ Adding more context to write agent's memory...
{
  "inference_results": [
    {
      "output": [
        {
          "name": "response",
          "result": "Scratchpad updated. Full content:\n- Previous query: Sales analysis for Q3 2025. User interested in regional breakdown."
        }
      ]
    }
  ]
}

⚠️ Note: Write and Read agents have separate memory spaces!


## Step 6: Create Agent with Read Capability

In [13]:
# Agent with ReadFromScratchPadTool
read_tools = [{
    "type": "ReadFromScratchPadTool",
    "parameters": {
        "persistent_notes": ""  # Empty = read all notes
    }
}]

read_agent_id = create_flow_agent(
    client, "Memory_Reader_Agent",
    "Agent that reads information from scratchpad memory",
    read_tools
)
print(f"✅ Read agent created: {read_agent_id}")

   Registering flow agent: Memory_Reader_Agent...
   ✓ Agent registered: lFuWiZsBLQ1mV2UNnyl8
✅ Read agent created: lFuWiZsBLQ1mV2UNnyl8


## Step 7: Test Reading from Memory

In [14]:
# ⚠️ This will be EMPTY because read_agent has a different memory space than write_agent!
parameters = {
    "persistent_notes": ""
}

print("📖 Reading from READ agent's scratchpad (which is separate)...")
response = execute_agent(client, read_agent_id, parameters)
print(json.dumps(response, indent=2))

print("\n💡 To share memory, use the SAME agent with both Read and Write tools!")
print("   See Step 8 for the correct pattern (Full_Memory_Agent)")

📖 Reading from READ agent's scratchpad (which is separate)...
{
  "inference_results": [
    {
      "output": [
        {
          "name": "response",
          "result": "Scratchpad is empty."
        }
      ]
    }
  ]
}

💡 To share memory, use the SAME agent with both Read and Write tools!
   See Step 8 for the correct pattern (Full_Memory_Agent)


## Step 8: Create Combined Read/Write Agent

In [15]:
# Agent with both read and write tools
memory_tools = [
    {
        "type": "ReadFromScratchPadTool",
        "parameters": {
            "persistent_notes": "Initial context: Customer service agent"
        }
    },
    {
        "type": "WriteToScratchPadTool",
        "parameters": {
            "return_history": True
        }
    }
]

memory_agent_id = create_flow_agent(
    client, "Full_Memory_Agent",
    "Agent with both read and write memory capabilities",
    memory_tools
)
print(f"✅ Full memory agent created: {memory_agent_id}")

   Registering flow agent: Full_Memory_Agent...
   ✓ Agent registered: lVuWiZsBLQ1mV2UN0CkT
✅ Full memory agent created: lVuWiZsBLQ1mV2UN0CkT


## Step 9: Simulate Conversation with Memory

In [16]:
# ⚠️ Demonstration: Each execution starts fresh, no memory accumulation

# Conversation turn 1
print("💬 Turn 1: User provides order number")
parameters = {
    "notes": "Customer order #12345. Status: Shipped. Expected delivery: Nov 12."
}
response = execute_agent(client, memory_agent_id, parameters)
print(json.dumps(response, indent=2))

# Conversation turn 2 - separate execution
print("\n💬 Turn 2: User asks follow-up (SEPARATE execution)")
parameters = {
    "notes": "User requested tracking number. Provided: TRK-789-XYZ"
}
response = execute_agent(client, memory_agent_id, parameters)
print(json.dumps(response, indent=2))

print("\n⚠️ Notice: Turn 2 only shows the second note, not both!")
print("💡 Each execute_agent() call starts with a fresh scratchpad.")
print("📝 For true persistence, you need external storage (database, OpenSearch index, etc.)")

💬 Turn 1: User provides order number
{
  "inference_results": [
    {
      "output": [
        {
          "name": "WriteToScratchPadTool",
          "result": "Scratchpad updated. Full content:\n- Customer order #12345. Status: Shipped. Expected delivery: Nov 12."
        }
      ]
    }
  ]
}

💬 Turn 2: User asks follow-up (SEPARATE execution)
{
  "inference_results": [
    {
      "output": [
        {
          "name": "WriteToScratchPadTool",
          "result": "Scratchpad updated. Full content:\n- User requested tracking number. Provided: TRK-789-XYZ"
        }
      ]
    }
  ]
}

⚠️ Notice: Turn 2 only shows the second note, not both!
💡 Each execute_agent() call starts with a fresh scratchpad.
📝 For true persistence, you need external storage (database, OpenSearch index, etc.)


## 🎓 Key Takeaways

### What We Learned:

1. **Scratchpad Tools**:
   - ✅ **WriteToScratchPadTool**: Store notes/context
   - ✅ **ReadFromScratchPadTool**: Retrieve stored information
   - ✅ **return_history**: Get all notes or just confirmation
   - ✅ **persistent_notes**: Initialize with context

2. **⚠️ CRITICAL: Memory Scope Limitations**:
   - ❌ **NOT persistent across `execute_agent()` calls** - each call starts fresh!
   - ❌ **Different agents have separate memories** - no shared scratchpad
   - ✅ **Memory works within a single agent execution** - for multi-step workflows
   - ✅ **Best for intra-workflow state** - not inter-conversation memory

3. **Memory Patterns**:
   ```python
   # Write pattern
   {
       "type": "WriteToScratchPadTool",
       "parameters": {
           "notes": "${parameters.notes}",
           "return_history": True
       }
   }
   
   # Read pattern
   {
       "type": "ReadFromScratchPadTool",
       "parameters": {
           "persistent_notes": "Initial context"
       }
   }
   ```

4. **Real-World Use Cases**:
   - ✅ **Complex Multi-Step Workflows**: Remember steps within ONE agent execution
   - ✅ **Tool Coordination**: Share data between tools in same workflow
   - ✅ **Decision Trees**: Track path taken through branches
   - ❌ **Multi-Turn Conversations**: Need external database (NOT scratchpad)
   - ❌ **User Session Memory**: Need application-level state management

5. **For True Persistent Memory, Use**:
   - 💾 External database (PostgreSQL, MongoDB, etc.)
   - 🔄 Application session management
   - 📊 OpenSearch index for storing conversation history
   - 🗃️ Custom memory management layer

6. **Best Practices**:
   - ✅ **Structured Notes**: Use consistent format
   - ✅ **Clear Context**: Include timestamps, IDs
   - ✅ **Right Tool for Job**: Don't use scratchpad for cross-execution memory
   - ⚠️ **Understand Limitations**: Know when scratchpad isn't enough

### Memory Scope Example:

```python
# ❌ This will NOT accumulate memory:
execute_agent(client, agent_id, {"notes": "First note"})  # Fresh scratchpad
execute_agent(client, agent_id, {"notes": "Second note"}) # Fresh scratchpad (loses first note!)

# ✅ Within single execution, tools can share memory:
# Agent workflow: WriteToScratchPad → ProcessData → ReadFromScratchPad
# All in ONE execute_agent() call = memory persists
```

---

## 🧹 Cleanup

## 💡 Demonstration: How Scratchpad Actually Works

The scratchpad is meant for **within-workflow** memory, where a single agent execution orchestrates multiple tool calls. Let's create an agent that demonstrates this properly.

### 🎯 Summary: Why Your Cell Didn't Return All Notes

**The Issue**: You expected `return_history=True` to return **all notes from previous executions**, but it only returned the current note.

**The Root Cause**: OpenSearch agent scratchpad memory is **execution-scoped**, not **persistent**:
- Each `execute_agent()` call creates a **new, empty scratchpad**
- Previous writes are **not retained** across separate executions
- Different agents have **completely separate** scratchpad spaces

**What Actually Happened**:
```
Step 4: execute_agent(write_agent_id, "First note")   → Scratchpad: ["First note"]
Step 5: execute_agent(write_agent_id, "Second note")  → Scratchpad: ["Second note"] ❌ Lost first!
Step 7: execute_agent(read_agent_id, "")              → Scratchpad: [] ❌ Different agent!
```

**Solution Options**:
1. **For intra-workflow memory**: Design agents with complex tool chains that execute in ONE call
2. **For persistent memory**: Use external database or OpenSearch index to store history
3. **For conversation memory**: Implement application-level session management

---

In [20]:
# Example: Testing if memory accumulates within same execution
import time

print("🔍 Testing memory behavior...")
print("\nTest 1: First write")
response1 = execute_agent(client, write_agent_id, {
    "notes": "First note at " + time.strftime("%H:%M:%S")
})
print(json.dumps(response1, indent=2))

print("\n" + "="*60)
print("Test 2: Second write (separate execution)")
response2 = execute_agent(client, write_agent_id, {
    "notes": "Second note at " + time.strftime("%H:%M:%S"),
    "return_history": True
})
print(json.dumps(response2, indent=2))

print("\n💡 Notice: Each execute_agent() call starts fresh - no memory accumulation!")

🔍 Testing memory behavior...

Test 1: First write
{
  "inference_results": [
    {
      "output": [
        {
          "name": "response",
          "result": "Scratchpad updated. Full content:\n- First note at 10:13:10"
        }
      ]
    }
  ]
}

Test 2: Second write (separate execution)
{
  "inference_results": [
    {
      "output": [
        {
          "name": "response",
          "result": "Scratchpad updated. Full content:\n- Second note at 10:13:10"
        }
      ]
    }
  ]
}

💡 Notice: Each execute_agent() call starts fresh - no memory accumulation!


In [ ]:
# # cleanup_resources(
# #     client=client,
# #     agent_ids=[write_agent_id, read_agent_id, memory_agent_id]
# # )
# # print("✅ Cleanup complete!")

## 🚀 Next Steps

- **AgentTool**: Combine memory with multi-agent systems
- **RAGTool**: Add memory to RAG conversations
- **MLModelTool**: Build stateful conversational AI

📚 [ScratchpadTools Documentation](https://opensearch.org/docs/latest/ml-commons-plugin/agents-tools/tools/scratchpad-tools/)